# Market basket analysis
<br/>
Apriori is a classic frequent-itemset mining algorithm that finds groups of items (itemsets) that appear together frequently in a transaction database, and then derives association rules from them.

### Core idea
Use the **Apriori property** — *if an itemset is frequent, then all its subsets are frequent* — to generate candidates level-by-level (size-1, size-2, …) and prune any candidate whose subset is known to be infrequent.
## Common use cases
- Market-basket analysis (what products are bought together) — cross-sell, store layout.
- Recommendation systems and product bundling.
- Web / clickstream pattern mining (pages visited together).
- Bioinformatics (frequent co-occurring genetic markers).
- Fraud or intrusion detection (frequent suspicious combinations).

## High-level step-by-step algorithm
1. **Input:** transaction database, `min_support` (threshold), optionally `min_confidence`.
2. **Scan 1:** generate all 1-item candidates ($C_1$), count supports, keep frequent ones $\rightarrow L_1$.
3. **For $k = 2, 3, ...$ until $L_{k-1}$ is empty:**
   - **Join:** form candidate $k$-itemsets $C_k$ by joining pairs of frequent $(k−1)$-itemsets from $L_{k-1}$.
   - **Prune:** remove any candidate in $C_k$ that has at least one $(k−1)$ subset not in $L_{k-1}$ (Apriori property).
   - **Count:** scan dataset to compute support for each candidate in $C_k$.
   - **Select:** let $L_k = \{c\in C_k | $support$(c) ≥ $min_support$\}$.
4. **Collect:** all $L_k$ form the set of frequent itemsets.
5. **Rule generation:** for each frequent itemset $F$ of size $\geq 2$, enumerate non-empty proper subsets $A$, form rule $A\rightarrow (F/A)$.
    <br/>Compute `confidence = support(F) / support(A)` and keep the rules with `confidence ≥ min_confidence`.
    <br/>Confidence is nothing but the conditional probability of $F$ w.r.t $A$, i.e, probability of $F$ occuring given that $A$ has already occured.
6. **Output:** frequent itemsets and association rules (optionally filtered by confidence).
***

In [1]:
def load_dataset():
    """Sample data for testing"""
    return [
        ['milk', 'bread', 'nuts', 'apple'],
        ['milk', 'bread', 'nuts'],
        ['milk', 'bread'],
        ['milk', 'bread', 'apple'],
        ['milk', 'bread', 'apple']
    ]

## Generate candidate itemsets of size $k$

In [2]:
from itertools import combinations

def create_candidates_from_dataset(dataset, k):
    candidates = set()
    for transaction in dataset:
        for combo in combinations(sorted(transaction), k):
            candidates.add(combo)
    return list(candidates)

def create_candidates_from_unique_items(items, k):
    """
    Generate candidate itemsets of size k from a list of unique items.
    
    Args:
        items (list/iterable): unique items (e.g., ['milk', 'bread', 'apple'])
        k (int): size of each candidate itemset
    
    Returns:
        list of tuples: candidate itemsets
    """
    return [tuple(sorted(c)) for c in combinations(items, k)]

## Calculate support for each candidate and filter based on `min_support`

In [3]:
def calculate_support(dataset, candidates, min_support = 0.5):
    item_count = {}
    total_transactions = len(dataset)

    for transaction in dataset:
        transaction_set = set(transaction)
        for candidate in candidates:
            if set(candidate).issubset(transaction_set):
                item_count[candidate] = item_count.get(candidate, 0) + 1

    # Filter with min_support
    frequent_items = []
    support_data = {}
    for item, count in item_count.items():
        support = count / total_transactions
        if support >= min_support:
            frequent_items.append(item)
        support_data[item] = support

    return frequent_items, support_data

## Function to generate antededents and consequents

In [4]:
from itertools import combinations

def generate_antecedent_consequent(itemset):
    """
    Given an itemset, generate all possible (antecedent, consequent) pairs.
    Each antecedent is a non-empty proper subset of the itemset.
    Consequent = itemset - antecedent
    """
    itemset = set(itemset)
    pairs = []
    
    # Generate all non-empty proper subsets
    for i in range(1, len(itemset)):
        for antecedent in combinations(itemset, i):
            antecedent = set(antecedent)
            consequent = itemset - antecedent
            pairs.append((antecedent, consequent))
    
    return pairs

## Generate unique items

In [5]:
dataset = load_dataset()
unique_items = create_candidates_from_dataset(dataset, 1)
unique_items = list(map(lambda x: x[0], unique_items))
unique_items

['apple', 'nuts', 'milk', 'bread']

## Generating item-sets example

In [6]:
## TODO: generate item-sets of 2
create_candidates_from_unique_items(unique_items, 2)

[('apple', 'nuts'),
 ('apple', 'milk'),
 ('apple', 'bread'),
 ('milk', 'nuts'),
 ('bread', 'nuts'),
 ('bread', 'milk')]

In [7]:
## TODO: generate item-sets of 3
create_candidates_from_unique_items(unique_items, 3)

[('apple', 'milk', 'nuts'),
 ('apple', 'bread', 'nuts'),
 ('apple', 'bread', 'milk'),
 ('bread', 'milk', 'nuts')]

## Generate antecedents and consequents example

In [8]:
print(f"Items: {unique_items}\n")
pairs = generate_antecedent_consequent(unique_items)

for antecedent, consequent in pairs:
    print(f"Antecedent: {antecedent}, Consequent: {consequent}")

Items: ['apple', 'nuts', 'milk', 'bread']

Antecedent: {'milk'}, Consequent: {'apple', 'nuts', 'bread'}
Antecedent: {'apple'}, Consequent: {'milk', 'nuts', 'bread'}
Antecedent: {'nuts'}, Consequent: {'milk', 'apple', 'bread'}
Antecedent: {'bread'}, Consequent: {'milk', 'apple', 'nuts'}
Antecedent: {'milk', 'apple'}, Consequent: {'nuts', 'bread'}
Antecedent: {'milk', 'nuts'}, Consequent: {'apple', 'bread'}
Antecedent: {'milk', 'bread'}, Consequent: {'apple', 'nuts'}
Antecedent: {'apple', 'nuts'}, Consequent: {'milk', 'bread'}
Antecedent: {'apple', 'bread'}, Consequent: {'milk', 'nuts'}
Antecedent: {'nuts', 'bread'}, Consequent: {'milk', 'apple'}
Antecedent: {'milk', 'apple', 'nuts'}, Consequent: {'bread'}
Antecedent: {'milk', 'apple', 'bread'}, Consequent: {'nuts'}
Antecedent: {'milk', 'nuts', 'bread'}, Consequent: {'apple'}
Antecedent: {'apple', 'nuts', 'bread'}, Consequent: {'milk'}


In [9]:
from pprint import pprint

support_map = {}

for k in range(1, len(unique_items) + 1):
    candidates_k = create_candidates_from_unique_items(unique_items, k)
    frequent_items_k, support_k = calculate_support(dataset, candidates_k)
    support_map[k] = {
        "Candidates": candidates_k,
        "Frequent items": frequent_items_k,
        "Support data": support_k
    }

get_support = lambda items: support_map[len(items)]["Support data"][items] 

pprint(support_map)

{1: {'Candidates': [('apple',), ('nuts',), ('milk',), ('bread',)],
     'Frequent items': [('apple',), ('milk',), ('bread',)],
     'Support data': {('apple',): 0.6,
                      ('bread',): 1.0,
                      ('milk',): 1.0,
                      ('nuts',): 0.4}},
 2: {'Candidates': [('apple', 'nuts'),
                    ('apple', 'milk'),
                    ('apple', 'bread'),
                    ('milk', 'nuts'),
                    ('bread', 'nuts'),
                    ('bread', 'milk')],
     'Frequent items': [('apple', 'milk'),
                        ('apple', 'bread'),
                        ('bread', 'milk')],
     'Support data': {('apple', 'bread'): 0.6,
                      ('apple', 'milk'): 0.6,
                      ('apple', 'nuts'): 0.2,
                      ('bread', 'milk'): 1.0,
                      ('bread', 'nuts'): 0.4,
                      ('milk', 'nuts'): 0.4}},
 3: {'Candidates': [('apple', 'milk', 'nuts'),
                    ('appl

## Apriori main algorithm

In [10]:
# TODO: Convert itemset to a sorted tuple for consistent dict keys.
def normalize_key(itemset):
    return tuple(sorted(itemset))

def generate_rules(min_confidence = 0.6):
    rules = []
    
    for item in support_map.keys():
        frequent_items = support_map[item]["Frequent items"]

        for items in frequent_items: 
            pairs = generate_antecedent_consequent(list(items))

            # print(pairs)

            normalized_item_key = normalize_key(items)
            combined_support = get_support(normalized_item_key)
        
            for antecedent, consequent in pairs:
                individual_support = get_support(normalize_key(antecedent))

                # print(combined_support, individual_support)
    
                confidence = combined_support / individual_support
                if confidence >= min_confidence:
                    rules.append({
                        "Antecedent": antecedent,
                        "Consequent": consequent,
                        "Confidence": round(confidence, 2)
                    })
    return sorted(rules, key = lambda record: record["Confidence"], reverse = True)

In [11]:
rules = generate_rules()
rules

[{'Antecedent': {'apple'}, 'Consequent': {'milk'}, 'Confidence': 1.0},
 {'Antecedent': {'apple'}, 'Consequent': {'bread'}, 'Confidence': 1.0},
 {'Antecedent': {'milk'}, 'Consequent': {'bread'}, 'Confidence': 1.0},
 {'Antecedent': {'bread'}, 'Consequent': {'milk'}, 'Confidence': 1.0},
 {'Antecedent': {'apple'}, 'Consequent': {'bread', 'milk'}, 'Confidence': 1.0},
 {'Antecedent': {'apple', 'milk'}, 'Consequent': {'bread'}, 'Confidence': 1.0},
 {'Antecedent': {'apple', 'bread'}, 'Consequent': {'milk'}, 'Confidence': 1.0},
 {'Antecedent': {'milk'}, 'Consequent': {'apple'}, 'Confidence': 0.6},
 {'Antecedent': {'bread'}, 'Consequent': {'apple'}, 'Confidence': 0.6},
 {'Antecedent': {'milk'}, 'Consequent': {'apple', 'bread'}, 'Confidence': 0.6},
 {'Antecedent': {'bread'}, 'Consequent': {'apple', 'milk'}, 'Confidence': 0.6},
 {'Antecedent': {'bread', 'milk'}, 'Consequent': {'apple'}, 'Confidence': 0.6}]

***